In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
print('plant classifier')

plant classifier


In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2026-01-04 12:49:17.702293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767530958.196183      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767530958.316986      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767530959.477890      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767530959.477933      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767530959.477936      55 computation_placer.cc:177] computation placer alr

In [4]:
import os

print(os.listdir("/kaggle/input"))
print(os.listdir("/kaggle/input/plantvillagedataset-unified"))

['plantvillagedataset-unified']
['PlantVillage_Unified copy']


In [5]:
import os

DATASET_ROOT = "/kaggle/input/plantvillagedataset-unified/PlantVillage_Unified copy"

TRAIN_DIR = os.path.join(DATASET_ROOT, "train")
VAL_DIR   = os.path.join(DATASET_ROOT, "val")
TEST_DIR  = os.path.join(DATASET_ROOT, "test")

print("Train folders:", os.listdir(TRAIN_DIR))
print("Val folders:", os.listdir(VAL_DIR))
print("Test folders:", os.listdir(TEST_DIR))


Train folders: ['unknown', 'tomato', 'pepper', 'potato']
Val folders: ['unknown', 'tomato', 'pepper', 'potato']
Test folders: ['unknown', 'tomato', 'pepper', 'potato']


In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


In [7]:
DATASET_ROOT = "/kaggle/input/plantvillagedataset-unified/PlantVillage_Unified copy"

TRAIN_DIR = os.path.join(DATASET_ROOT, "train")
VAL_DIR   = os.path.join(DATASET_ROOT, "val")
TEST_DIR  = os.path.join(DATASET_ROOT, "test")

print("Train:", os.listdir(TRAIN_DIR))
print("Val:", os.listdir(VAL_DIR))
print("Test:", os.listdir(TEST_DIR))


Train: ['unknown', 'tomato', 'pepper', 'potato']
Val: ['unknown', 'tomato', 'pepper', 'potato']
Test: ['unknown', 'tomato', 'pepper', 'potato']


In [8]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25   # EarlyStopping will stop earlier
SEED = 42


In [9]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    shear_range=0.15,
    brightness_range=(0.7, 1.3),
    horizontal_flip=True,
    fill_mode="nearest"
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)


In [10]:
train_data = train_gen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    seed=SEED
)

val_data = val_gen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    seed=SEED
)

test_data = test_gen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

print("Class indices:", train_data.class_indices)


Found 21088 images belonging to 4 classes.
Found 3437 images belonging to 4 classes.
Found 3160 images belonging to 4 classes.
Class indices: {'pepper': 0, 'potato': 1, 'tomato': 2, 'unknown': 3}


In [11]:
labels = train_data.classes
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)

class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)


Class weights: {0: np.float64(2.13010101010101), 1: np.float64(2.449814126394052), 2: np.float64(0.3703807784178727), 3: np.float64(2.3673102828917827)}


In [12]:
base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights="imagenet"
)

base_model.trainable = False  # VERY IMPORTANT

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation="softmax")
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


I0000 00:00:1767531044.982723      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1767531044.986582      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,592,068 (9.89 MB)

 Trainable params: 331,524 (1.26 MB)

 Non-trainable params: 2,260,544 (8.62 MB)

In [13]:
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=3,
        min_lr=1e-6
    ),
    ModelCheckpoint(
        "plant_classifier_best.keras",
        monitor="val_loss",
        save_best_only=True
    )
]


In [14]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25


I0000 00:00:1767531086.001241     149 service.cc:152] XLA service 0x7c022c012030 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767531086.001277     149 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1767531086.001281     149 service.cc:160]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1767531087.232149     149 cuda_dnn.cc:529] Loaded cuDNN version 91002
2026-01-04 12:51:36.512077: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-04 12:51:36.649062: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
I0000 00:00:1767531099.057213     149 device_co

659/659 ━━━━━━━━━━━━━━━━━━━━ 0s 626ms/step - accuracy: 0.7073 - loss: 0.6119

2026-01-04 12:59:03.682235: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-04 12:59:03.828857: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-04 12:59:03.965466: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


659/659 ━━━━━━━━━━━━━━━━━━━━ 467s 679ms/step - accuracy: 0.7075 - loss: 0.6115 - val_accuracy: 0.9572 - val_loss: 0.1209 - learning_rate: 1.0000e-04
Epoch 2/25
659/659 ━━━━━━━━━━━━━━━━━━━━ 291s 441ms/step - accuracy: 0.9269 - loss: 0.1453 - val_accuracy: 0.9761 - val_loss: 0.0648 - learning_rate: 1.0000e-04
Epoch 3/25
659/659 ━━━━━━━━━━━━━━━━━━━━ 287s 436ms/step - accuracy: 0.9506 - loss: 0.1053 - val_accuracy: 0.9793 - val_loss: 0.0574 - learning_rate: 1.0000e-04
Epoch 4/25
659/659 ━━━━━━━━━━━━━━━━━━━━ 291s 442ms/step - accuracy: 0.9547 - loss: 0.0916 - val_accuracy: 0.9820 - val_loss: 0.0546 - learning_rate: 1.0000e-04
Epoch 5/25
659/659 ━━━━━━━━━━━━━━━━━━━━ 286s 433ms/step - accuracy: 0.9602 - loss: 0.0760 - val_accuracy: 0.9843 - val_loss: 0.0465 - learning_rate: 1.0000e-04
Epoch 6/25
659/659 ━━━━━━━━━━━━━━━━━━━━ 289s 438ms/step - accuracy: 0.9661 - loss: 0.0646 - val_accuracy: 0.9834 - val_loss: 0.0447 - learning_rate: 1.0000e-04
Epoch 7/25
659/659 ━━━━━━━━━━━━━━━━━━━━ 297s 450ms/

In [15]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")


98/99 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.9949 - loss: 0.0158

2026-01-04 15:12:46.025298: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2026-01-04 15:12:46.171550: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


99/99 ━━━━━━━━━━━━━━━━━━━━ 35s 355ms/step - accuracy: 0.9949 - loss: 0.0159
Test Accuracy: 0.9940


In [12]:
model.save("/kaggle/working/plant_classifier.keras")


NameError: name 'model' is not defined

In [11]:
import tensorflow as tf

model = tf.keras.models.load_model("plant_classifier.keras", compile=False)

# Save as SavedModel
model.save("plant_classifier_tf", save_format="tf")


ValueError: File not found: filepath=plant_classifier.keras. Please ensure the file is an accessible `.keras` zip file.

In [3]:
!find / -name "plant_classifier.keras" 2>/dev/null


In [10]:
import os
print(os.listdir("/kaggle/working"))


['.virtual_documents']
